In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# -------------------------------------------------------
# Build 1D Kitaev chain BdG Hamiltonian (spinless p-wave)
# Basis: (c1,...,cN, c1^†,...,cN^†)  -> dimension 2N
# -------------------------------------------------------
def build_kitaev_bdg(N, t, mu, Delta):
    """
    N     : number of sites
    t     : hopping amplitude
    mu    : chemical potential
    Delta : p-wave pairing between nearest neighbours
    """
    h = np.zeros((N, N), dtype=complex)       # normal (tight-binding) part
    Delta_mat = np.zeros((N, N), dtype=complex)  # pairing matrix

    for i in range(N):
        # on-site term
        h[i, i] = -mu

        # nearest-neighbor hopping + pairing
        if i < N - 1:
            # hopping
            h[i, i + 1] = -t
            h[i + 1, i] = -t

            # p-wave pairing is antisymmetric in i,j
            Delta_mat[i, i + 1] = Delta
            Delta_mat[i + 1, i] = -Delta

    # BdG Hamiltonian:
    # H = [[ h,           Δ      ],
    #      [ -Δ* ,     -h*      ]]
    H = np.block([
        [h,                  Delta_mat],
        [-Delta_mat.conj(),  -h.conj()]
    ])

    return H

# -------------------------------------------------------
# Find the lowest-|E| eigenstates (near zero energy)
# -------------------------------------------------------
def find_zero_modes(H, num_modes=2):
    # Full diagonalization (fine for moderate N)
    E, V = np.linalg.eigh(H)
    # Sort by |E| to find states closest to zero
    idx = np.argsort(np.abs(E))[:num_modes]
    return E[idx], V[:, idx]   # eigenvalues, eigenvectors (columns)

# -------------------------------------------------------
# Form spatially localized Majorana modes
# from the two almost-degenerate zero-energy eigenvectors
# -------------------------------------------------------
def majorana_from_zero_modes(V_zero):
    """
    V_zero: 2N x 2 array of zero-mode eigenvectors
    Returns two Majorana modes (gamma_L, gamma_R)
    """
    psi1 = V_zero[:, 0]
    psi2 = V_zero[:, 1]

    # Orthogonal Majorana combinations
    gamma_L = (psi1 + psi2) / np.sqrt(2.0)
    gamma_R = 1j * (psi1 - psi2) / np.sqrt(2.0)

    return gamma_L, gamma_R

# -------------------------------------------------------
# Compute local density on each site:
#  ρ(i) = |u_i|^2 + |v_i|^2
# -------------------------------------------------------
def local_density(gamma, N):
    u = gamma[:N]     # particle part
    v = gamma[N:]     # hole part
    return np.abs(u)**2 + np.abs(v)**2

# -------------------------------------------------------
# Main routine: build chain, find Majoranas, and plot
# -------------------------------------------------------
def plot_majorana_zero_modes(N=100, t=1.0, mu=0.0, Delta=1.0):
    """
    Choose parameters in the topological phase: |mu| < 2t
    e.g. t=1, Delta=1, mu=0 is deep in the topological regime.
    """
    H = build_kitaev_bdg(N, t, mu, Delta)

    # Find the two states closest to zero energy
    E_zero, V_zero = find_zero_modes(H, num_modes=2)
    print("Lowest |E| eigenvalues (should be ~0 in topological phase):")
    print(E_zero)

    # Build localized Majorana modes
    gamma_L, gamma_R = majorana_from_zero_modes(V_zero)

    # Spatial profiles
    dens_L = local_density(gamma_L, N)
    dens_R = local_density(gamma_R, N)
    sites = np.arange(1, N + 1)

    # Plot
    plt.figure(figsize=(7, 4))
    plt.plot(sites, dens_L, marker='o', linestyle='-', label='Left Majorana')
    plt.plot(sites, dens_R, marker='s', linestyle='-', label='Right Majorana')
    plt.xlabel('Site index i', fontsize=12)
    plt.ylabel(r'Local density $|\gamma(i)|^2$', fontsize=12)
    plt.title('Kitaev Chain Majorana Zero Modes (1D, open boundary)', fontsize=13)
    plt.legend()
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    # You can tweak these to explore trivial vs topological phases:
    # Topological: |mu| < 2t
    plot_majorana_zero_modes(N=100, t=1.0, mu=0.0, Delta=1.0)
    # For comparison, try e.g. mu=3.0 (trivial phase, no edge modes)
    # plot_majorana_zero_modes(N=100, t=1.0, mu=3.0, Delta=1.0)
